# Import Libriaries

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import html5lib
import json
import time

import datetime
import feedparser
import ftplib
import io
import requests
import requests_html
import yahoo_fin.stock_info as si
from yahoo_fin import options

# Import API Key 

In [2]:
# API Key 
demo = "Enter your API KEY HERE"

# Company Tickers
tickers = ['MSFT','ATVI','BAC','NFLX','DIS']

# Index Tickers
index_tickers = ['^GSPC','^STOXX','^FTSE']

# Create empty dictionaries to store data for each company
data = {}
index_data = {}
BS_Fin = {}
IS_Fin = {}
CF_Fin = {}

# Import Financial Statements Data Function

In [3]:
def fin_statements (company,freq,num_of_per):
    '''
    Descreption : obtain companies financial statements based on provided frequency (Annual/Quarter) 
    and a specific number of periods.
    
    function parameters:
    
    company: company ticker name
    freq: frequency of financial statements either annual or quarter
    num_of_per: number of periods either years or quarters 

    '''
    global BS_Fin
    global IS_Fin
    global CF_Fin
         
    try:
     
        BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?period={freq}&limit={num_of_per}&apikey={demo}').json()
        BS = pd.DataFrame(BS)
        BS_Fin[company] = BS
        time.sleep(5)
        
        print('BS_completed for: '  " " + company)
        
    except:
        
        print('ERROR_in_BS for: ' " " + company)
        
    try:

        IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?period={freq}&limit={num_of_per}&apikey={demo}').json()
        IS = pd.DataFrame(IS)
        IS_Fin[company] = IS
        time.sleep(5)
        
        print('IS_completed for: ' " " + company)
        
    except:
        
        print('ERROR_in_IS for: ' " " + company)
        
    try:
        CF= requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{company}?period={freq}&limit={num_of_per}&apikey={demo}').json()
        CF = pd.DataFrame(CF)
        CF_Fin[company] = CF
        time.sleep(5)
        
        print('CF_completed for: ' " " + company)
        
    except:
        
        print('ERROR_in_CF for: ' " " + company)

# Import Stock Data Function  

In [4]:
def get_data (company,stock_start_date,stock_end_date):
    
    global data
    
    try: 
        # Transform the Json String to Pandas DataFrame
        
        stocks = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{company}?from={stock_start_date}&to={stock_end_date}&apikey={demo}').json()
        stocks = stocks['historical']
        stock_prices_data = pd.DataFrame.from_dict(stocks)
        data[company] = stock_prices_data
        data[company]['ticker']=company
        
        # Sort Values by Date
        data[company] = data[company].sort_values(by="date")
        
        # Set Date as Index 
        data[company] = data[company].set_index('date')
        time.sleep(5)
            
        print('get_data_function_completed for: ' + company)
        print(company + ": Data Added Succesfully")
       
 
    except:
        
        print('ERROR_get_data_function_NOT_completed for: ' + company)
        print('ERROR something went wrong with Data : ' + company)

# Index Import Data

In [5]:
def index_collection_function(index,index_start_date,index_end_date):
    
    global index_data
    
    try:
        # Get Daily Index Price Data
        index_data[index] = si.get_data(index,start_date=index_start_date,end_date=index_end_date)
        
        print('index_collection_function_completed for:' + index)
        
    except:
        
        print('index_collection_function_NOT_completed for:' + index)

# Data collection

In [6]:
# Set parameters for companies research 
freq = 'annual' # Parameter 1 frequency
num_of_per = 5 # Parameter 2 number of periods

# Stock tickers period selection 
stock_start_date = '2017-04-20' #Y-M-D
stock_end_date = '2022-04-19' #Y-M-D

for company in tickers:
    
    # Import Financial statements Data
    try:

        fin_statements (company,freq,num_of_per)  

        print('fin_statements_function for: ' + company + '  Added')

    except:

           print('ERROR_fin_statements_function_NOT_Finalized for: ' + company)
            
    # Import Stock Data        
    try:

        get_data (company,stock_start_date,stock_end_date)  

        print('stock_data for: ' + company + '  Added')

    except:

           print('ERROR_stock_data_NOT_Finalized for: ' + company)


BS_completed for:  MSFT
IS_completed for:  MSFT
CF_completed for:  MSFT
fin_statements_function for: MSFT  Added
get_data_function_completed for: MSFT
MSFT: Data Added Succesfully
stock_data for: MSFT  Added
BS_completed for:  ATVI
IS_completed for:  ATVI
CF_completed for:  ATVI
fin_statements_function for: ATVI  Added
get_data_function_completed for: ATVI
ATVI: Data Added Succesfully
stock_data for: ATVI  Added
BS_completed for:  BAC
IS_completed for:  BAC
CF_completed for:  BAC
fin_statements_function for: BAC  Added
get_data_function_completed for: BAC
BAC: Data Added Succesfully
stock_data for: BAC  Added
BS_completed for:  NFLX
IS_completed for:  NFLX
CF_completed for:  NFLX
fin_statements_function for: NFLX  Added
get_data_function_completed for: NFLX
NFLX: Data Added Succesfully
stock_data for: NFLX  Added
BS_completed for:  DIS
IS_completed for:  DIS
CF_completed for:  DIS
fin_statements_function for: DIS  Added
get_data_function_completed for: DIS
DIS: Data Added Succesfully
s

In [7]:
# Index tickers period selection
index_start_date = '04/20/2017' #M-D-Y
index_end_date = '04/19/2022' #M-D-Y
    
for index in index_tickers: 
    
    try:
        
        index_collection_function(index,index_start_date,index_end_date)        

        print('Index: ' + index + '  Added')

    except:

           print('ERROR Index: ' + index + ' Not Added')
    
    

index_collection_function_completed for:^GSPC
Index: ^GSPC  Added
index_collection_function_completed for:^STOXX
Index: ^STOXX  Added
index_collection_function_completed for:^FTSE
Index: ^FTSE  Added


# Concatinate the financial statements and Stock Data

In [8]:
# Combine Financial Statements 
BS_Final = pd.concat(BS_Fin)
IS_Final = pd.concat(IS_Fin)
CF_Final = pd.concat(CF_Fin)

BS_Final['Fin_Year'] = BS_Final['calendarYear']
IS_Final['Fin_Year'] = IS_Final['calendarYear']
CF_Final['Fin_Year'] = CF_Final['calendarYear']

BS_Final = BS_Final.reset_index().set_index(['level_0','Fin_Year']).sort_index().sort_values(['symbol']).drop(columns = ['level_1'])
IS_Final = IS_Final.reset_index().set_index(['level_0','Fin_Year']).sort_index().sort_values(['symbol']).drop(columns = ['level_1'])
CF_Final = CF_Final.reset_index().set_index(['level_0','Fin_Year']).sort_index().sort_values(['symbol']).drop(columns = ['level_1'])

In [9]:
# Combine Stock Data  
data_Final = pd.concat(data)
monthly_data_Final = data_Final.reset_index().drop(columns = ['level_0'])
monthly_data_Final['Date'] = monthly_data_Final['date']
monthly_data_Final['Date'] = pd.to_datetime(monthly_data_Final['Date'])
monthly_data_Final = monthly_data_Final.set_index(['Date']).sort_values(['ticker','date'])
monthly_data_Final = monthly_data_Final.groupby('ticker').resample('M').mean()
monthly_data_Final = monthly_data_Final.reset_index()
monthly_data_Final = monthly_data_Final.set_index(['Date'])

In [10]:
# Combine Index Data
index_data_dataframe = pd.concat(index_data)
index_data_dataframe = index_data_dataframe.reset_index().rename(columns={'level_1':'date'})
index_data_dataframe['date'] = index_data_dataframe['date'].dt.strftime('%Y-%m-%d')
index_data_dataframe = index_data_dataframe.set_index(['level_0','date'])

In [11]:
# Export data files into Excel Files
BS_Final.to_excel('BS_Final.xlsx',sheet_name='BS_Final')
IS_Final.to_excel('IS_Final.xlsx',sheet_name='IS_Final')
CF_Final.to_excel('CF_Final.xlsx',sheet_name='CF_Final')
data_Final.to_excel('data_Final.xlsx',sheet_name='data_Final')
index_data_dataframe.to_excel('index_data_dataframe.xlsx',sheet_name='index_data')